In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from itertools import product
import math
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as accuracy
import graphviz 
import psycopg2

sns.set(style = "ticks", color_codes = True)
%matplotlib inline 

%load_ext autoreload
%autoreload 2

from __future__ import division
import pandas as pd
import numpy as np
from sklearn import preprocessing, cross_validation, svm, metrics, tree, decomposition, svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier, OrthogonalMatchingPursuit, RandomizedLogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import ParameterGrid
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
import random
import matplotlib.pyplot as plt
from scipy import optimize
import time
import seaborn as sns
import ML_functions as mlf
from sklearn.metrics import recall_score

from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings('ignore')

#Referenced and modified Rayid's magic loop 
#Magic loop https://github.com/rayidghani/magicloops 
#Used stackoverflow for guidance on visualizations and dataframe operations


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# ssh variables
host = 'postgres.dssg.io'
localhost = 'mlclass.dssg.io'
ssh_username = 'sghosewi' 
ssh_private_key = '~/.ssh/sghosewi'

# database variables
user='jocodssg_students'
password='aibaecighoobeeba'
database='jocodssg'


In [3]:
# conn_string = "host='127.0.0.1' port='3308' dbname='jocodssg' user='jocodssg_students' password='aibaecighoobeeba'"
conn = psycopg2.connect(host='127.0.0.1', port='3333', database='jocodssg', user='jocodssg_students', password='aibaecighoobeeba')
cur = conn.cursor()

In [4]:
def sql_to_df(q):
    with SSHTunnelForwarder(
        (host, 3333),
        ssh_username=ssh_username,
        ssh_private_key=ssh_private_key,
        remote_bind_address=(localhost, 22)
    ) as server:
        conn = psycopg2.connect(host=localhost,
                               port=server.local_bind_port,
                               user=user,
                               passwd=password,
                               db=database)

        return pd.read_sql_query(q, conn)

In [5]:
def run_query(sql_query, conn):
    try:
        cur = conn.cursor()
        cur.execute(sql_query)
        column_names = [desc[0] for desc in cur.description]
        res = cur.fetchall()
        res_named = [tuple(column_names)] + res
        cur.close()
        res_df = pd.DataFrame(res)
        res_df.columns = column_names
        return res_df
    except psycopg2.Error as e:
        print(e)
        conn.rollback()

In [6]:
def run_query_save(sql_query, conn, filename):
    try:
        cur = conn.cursor()
        outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(sql_query)
        with open(filename, 'w') as f:
            cur.copy_expert(outputquery, f)
        cur.close()
    except psycopg2.Error as e:
        print(e)
        conn.rollback()

In [ ]:
mental_health_query='''
with mental_health as (
    select mh.dedupe_id,
    mh.patid, 
    DATE_PART('day', mh.dschrg_date::timestamp - mh.admit_date::timestamp)::int as mh_stay_days, 
    mh.dob,
    mh.program, 
    mh.pri_dx_value, 
    disch.admit_date, 
    dschg_date, 
    discharge_reason, 
    diag.dx_description, 
    diag.dx_date 
    from clean.jocomentalhealth_hashed mh
    left outer join clean.jocomentalhealthdiagnoses diag 
    on mh.patid = diag.patid 
    left outer join clean.jocomentalhealthdischarges disch 
    on diag.patid = disch.patid),
person_info as (
    select indiv.dedupe_id,
    indiv.sex,
    indiv.dob,
    extract(year from age(timestamp '2010-01-01', indiv.dob::timestamp))::int as age_calced,
    indiv.race,
    inm.mar_stat,
    inm.city,
    inm.zip,
    inm.arr_agency,
    inm.arrest_dt,
    inm.rel_date,
    extract(year from age(inm.arrest_dt::timestamp, inm.dob::timestamp))::int as arrest_age,
    inm.bk_dt,
    DATE_PART('day', inm.bk_dt::timestamp - inm.arrest_dt::timestamp)::int as unbooked_days,
    DATE_PART('day', inm.rel_date::timestamp - inm.bk_dt::timestamp)::int as stay_days
    from clean.individuals indiv left outer join clean.jocojims2inmatedata inm on indiv.dedupe_id = inm.dedupe_id
    )
select person.*,
mht.patid, 
mht.mh_stay_days, 
mht.program, 
CASE WHEN mht.pri_dx_value IS NOT NULL
THEN 1
ELSE 0
END as mh_diagnosis, 
mht.admit_date, 
mht.dschg_date, 
mht.discharge_reason, 
mht.dx_description, 
mht.dx_date
from person_info person left outer join mental_health mht 
on person.dedupe_id = mht.dedupe_id and person.dob::timestamp = mht.dob::timestamp
'''

In [ ]:
run_query_save(mental_health_query, conn, 'mental_health_data1.csv')

In [ ]:
mental_health_query += ';'
mh_df = run_query(mental_health_query, conn)

In [ ]:
mh_df = pd.read_csv('mental_health_data.csv', header = 0)

In [ ]:
mh_df.columns

In [ ]:
def view_histogram(df, bins = 10, normed = False):
    pd.cut(df , bins).value_counts(normalize=normed).sort_index().plot(kind='bar')

In [ ]:
view_histogram(mh_df.age_calced, bins = [0,10,14,18,22,26,30,40,50,60,70,80], normed = True)

In [ ]:
mh_df.columns

In [ ]:
len(mh_df['dedupe_id'].unique())

In [ ]:
mh_df.groupby(['dedupe_id']).count()

In [ ]:
mh_df['mh_stay_days']

## Mental Health Stay Days Histogram

In [ ]:
view_histogram(mh_df.mh_stay_days, bins = [0,50,100,150,200,250,300,350,400,500,1000, 2000, 3000], normed = False)

In [ ]:
query = ''' select * from clean.jocojimsperson_hashed limit 10;'''

In [ ]:
run_query(query, conn)

In [ ]:
mh_df.columns

In [3]:
mh_df1 = pd.read_csv('vf1.csv', header = 0)
#mh_df1 = pd.read_csv('/Users/saptarshighose/Downloads/vf1.csv', header = 0)

In [7]:
mh_df1.columns

Index(['dedupe_id', 'sex', 'dob', 'age_calced', 'race', 'mar_stat', 'city',
       'zip', 'rel_date', 'arr_agency', 'arrest_dt', 'arrest_age', 'bk_dt',
       'unbooked_days', 'stay_days', 'patid', 'mh_stay_days', 'program',
       'pri_dx_value', 'admit_date', 'dschg_date', 'discharge_reason',
       'dx_description', 'dx_date'],
      dtype='object')

In [8]:
mandatory_counseling_list = ["DUI/DWI",
"COMMUNITY CORRECITONS",
"JUVENILE JUSTICE AUTHORITY",
"COMMUNITY CORRECTIONS",
"CRISIS STABILIZATION UNIT",
"STATE MENTAL RETARDATION HOSPITAL",
"POLICE",
"COMMUNITY SERVICE PROGRAMS",
"COMMUNITY DEVELOPMENTAL DISABILITY ORGANIZATION",
"DCF VR DISABILITY DETERMINATION",
"ADOPTION PRIVATIZATION CONTRACT",
"PENAL SYSTEM",
"RESIDENTIAL FACILITY (ADULT GRP HOME)",
"DCF - AREA OFFICE",
"DIVERSIONARY PROGRAM",
"PROBATION",
"JUV.CORRECTION FACILITY (TJCF, LJCF, AJCF, BJCF)",
"NURSING FACILITY FOR MENTAL HEALTH (NFMH)",
"JUV.CORRETION FACILITY(TJCF, LJCF, AJCF, BJCF)",
"FOSTER CARE PRIVATIZATION CONTRACT",
"PAROLE",
"COURT"]

In [10]:
# mh_df2['referral_dummy'] = mh_df1['refferal_source']

In [4]:
def turn_to_1_0(df, b):
    for col in b:
        df[col] = df[col].apply(lambda x: 1 if x in mandatory_counseling_list else 0)

In [5]:
# turn_to_1_0(mh_df1, ["referral_dummy"])

In [6]:
# diag.head(50)
# all_diag = diag.pri_dx_code.unique()
# all_codes = list(all_diag)
# dsm17 = []
# for code in all_diag:
#     if any(char.isalpha() for char in code):
#         dsm17.append(code)
#         all_codes.remove(code)

In [7]:
mh_df1['pri_dx_value'] = mh_df1['pri_dx_value'].str.upper()

In [8]:
mh_df1['pri_dx_value'].unique()

array([nan, 'DYSTHYMIA', 'UNSPECIFIED AFFECTIVE PSYCHOSIS',
       'PROLONGED POST TRAUMATIC STRESS DISORDER',
       'ADJUSTMENT DISORDER W/MIXED DISTURBANCE OF EMOTIONS & CONDCT',
       'UNSPECIFIED PERSONALITY DISORDER',
       'MAJOR DEPRESSIVE DISORDER, RECURRENT, MODERATE',
       'UNSP PSYCHOSIS NOT DUE TO A SUBSTANCE OR KNOWN PHYSIOL COND',
       'IMPULSE CONTROL DISORDER, UNSPECIFIED',
       'AGORAPHOBIA WITH PANIC ATTACKS',
       'MAJOR DEPRESSIVE DISORDER, RECURRENT SEVERE WITHOUT PSYCHOTIC FEATURES',
       'BIPOLAR AFFECTIVE DISORDER, UNSPECIFIED',
       'SCHIZOAFFECTIVE DISORDER, BIPOLAR TYPE', 'BIPOLAR II DISORDER',
       'MAJOR DEPRESS. DISORDER, RECURRENT, IN FULL REMISSION',
       'ANXIETY STATE, UNSPECIFIED', 'BIPOLAR DISORDER, UNSPECIFIED',
       'DEPRESSIVE DISORDER NOS',
       'MAJOR DEPRESSIVE DISORDER, SINGLE EPISODE, MODERATE',
       'GENERALIZED ANXIETY DISORDER',
       'BIPOLAR DISORDER, MIXED, SEVERE, W/O PSYCHOTIC',
       'BIPOLAR DISORD, CRNT E

In [9]:
mh_df1['anxiety_dummy'] = mh_df1['pri_dx_value']
mh_df1['depression_dummy'] = mh_df1['pri_dx_value']
mh_df1['psychotic_dummy'] = mh_df1['pri_dx_value']
mh_df1['ptsd_dummy'] = mh_df1['pri_dx_value']
mh_df1['opps_dummy'] = mh_df1['pri_dx_value']
mh_df1['drugs_dummy'] = mh_df1['pri_dx_value']

mh_df1.head()

,dedupe_id,sex,dob,age_calced,race,mar_stat,city,zip,rel_date,arr_agency,...,dschg_date,discharge_reason,dx_description,dx_date,anxiety_dummy,depression_dummy,psychotic_dummy,ptsd_dummy,opps_dummy,drugs_dummy
0,2.0,MALE,1955-08-25,54.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,MALE,1962-03-18,47.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,FEMALE,1976-09-07,33.0,WHITE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,FEMALE,1980-05-27,29.0,WHITE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,DYSTHYMIC DISORDER ...,2012-09-07,DYSTHYMIA,DYSTHYMIA,DYSTHYMIA,DYSTHYMIA,DYSTHYMIA,DYSTHYMIA
4,6.0,MALE,1937-08-14,72.0,WHITE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def mh_to_cat(x, words_list):
    for word in words_list:
        if word in str(x):
            return  1
        else:
            return 0

In [11]:
anxiety_words_list = ['ANXIETY', 'ANX']


In [12]:
mh_df1['anxiety_dummy'] = mh_df1['pri_dx_value'].apply(mh_to_cat, args = (anxiety_words_list,))

In [13]:
depression_words_list = ['DEPRESSIVE']
mh_df1['depression_dummy'] = mh_df1['pri_dx_value'].apply(mh_to_cat, args = (depression_words_list,))

In [14]:
psychotic_words_list = ['BIPOLAR', 'SCHIZ']
mh_df1['psychotic_dummy'] = mh_df1['pri_dx_value'].apply(mh_to_cat, args = (psychotic_words_list,))

In [15]:
ptsd_words_list = ['TRAUMATIC', 'PTSD']
mh_df1['ptsd_dummy'] = mh_df1['pri_dx_value'].apply(mh_to_cat, args = (psychotic_words_list,))

In [16]:
opps_words_list = ['OPPOSITIONAL']
mh_df1['opps_dummy'] = mh_df1['pri_dx_value'].apply(mh_to_cat, args = (opps_words_list,))

In [17]:
drugs_words_list = ['DRUG','SUBST', 'CANNA', 'OPOI', 'AMPH', 'STIM', 'COCAI', 'INHAL', 'ALCOHOL']
mh_df1['drugs_dummy'] = mh_df1['pri_dx_value'].apply(mh_to_cat, args = (drugs_words_list,))

In [18]:
mh_df1.head(100)

,dedupe_id,sex,dob,age_calced,race,mar_stat,city,zip,rel_date,arr_agency,...,dschg_date,discharge_reason,dx_description,dx_date,anxiety_dummy,depression_dummy,psychotic_dummy,ptsd_dummy,opps_dummy,drugs_dummy
0,2.0,MALE,1955-08-25,54.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
1,3.0,MALE,1962-03-18,47.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2,4.0,FEMALE,1976-09-07,33.0,WHITE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
3,5.0,FEMALE,1980-05-27,29.0,WHITE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,DYSTHYMIC DISORDER ...,2012-09-07,0,0,0,0,0,0
4,6.0,MALE,1937-08-14,72.0,WHITE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
5,7.0,FEMALE,1951-11-07,58.0,WHITE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
6,8.0,FEMALE,1977-12-22,32.0,WHITE,S,KANSAS CITY,66102,2003-04-16,KS0460600,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
7,8.0,FEMALE,1977-12-22,32.0,WHITE,NaN,KANSAS CITY,64131,2001-08-11,KS0460600,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
8,8.0,FEMALE,1977-12-22,32.0,WHITE,S,KANSAS CITY,66102,2003-02-03,KS0460600,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
9,8.0,FEMALE,1977-12-22,32.0,WHITE,S,KANSAS CITY,66102,2002-03-27,KS0460600,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


In [19]:
def count_previous_mh_services(df):
    count_df = mh_df.groupby('dedupe_id').dedupe_id.nunique()
    return count_df

count_previous_mh_services(mh_df1)

#try grouping by dedupe_od/ summing 

NameError: name 'mh_df' is not defined

In [45]:

NOTEBOOK = 0

def define_clfs_params(grid_size):
    """Define defaults for different classifiers.
    Define three types of grids:
    Test: for testing your code
    Small: small grid
    Large: Larger grid that has a lot more parameter sweeps
    """

    clfs = {'RF': RandomForestClassifier(n_estimators=50, n_jobs=-1),
        'ET': ExtraTreesClassifier(n_estimators=10, n_jobs=-1, criterion='entropy'),
        'AB': AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME", n_estimators=200),
        'LR': LogisticRegression(penalty='l1', C=1e5),
        'SVM': svm.SVC(kernel='linear', probability=True, random_state=0),
        'GB': GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10),
        'NB': GaussianNB(),
        'DT': DecisionTreeClassifier(),
        'SGD': SGDClassifier(loss="hinge", penalty="l2"),
        'KNN': KNeighborsClassifier(n_neighbors=3) 
            }

    large_grid = { 
    'RF':{'n_estimators': [1,10,100,1000,10000], 'max_depth': [1,5,10,20,50,100], 'max_features': ['sqrt','log2'],'min_samples_split': [2,5,10], 'n_jobs': [-1]},
    'LR': { 'penalty': ['l1','l2'], 'C': [0.00001,0.0001,0.001,0.01,0.1,1,10]},
    'SGD': { 'loss': ['hinge','log','perceptron'], 'penalty': ['l2','l1','elasticnet']},
    'ET': { 'n_estimators': [1,10,100,1000,10000], 'criterion' : ['gini', 'entropy'] ,'max_depth': [1,5,10,20,50,100], 'max_features': ['sqrt','log2'],'min_samples_split': [2,5,10], 'n_jobs': [-1]},
    'AB': { 'algorithm': ['SAMME', 'SAMME.R'], 'n_estimators': [1,10,100,1000,10000]},
    'GB': {'n_estimators': [1,10,100,1000,10000], 'learning_rate' : [0.001,0.01,0.05,0.1,0.5],'subsample' : [0.1,0.5,1.0], 'max_depth': [1,3,5,10,20,50,100]},
    'NB' : {},
    'DT': {'criterion': ['gini', 'entropy'], 'max_depth': [1,5,10,20,50,100],'min_samples_split': [2,5,10]},
    'SVM' :{'C' :[0.00001,0.0001,0.001,0.01,0.1,1,10],'kernel':['linear']},
    'KNN' :{'n_neighbors': [1,5,10,25,50,100],'weights': ['uniform','distance'],'algorithm': ['auto','ball_tree','kd_tree']}
           }
    
    small_grid = { 
    'RF':{'n_estimators': [10,100], 'max_depth': [5,50], 'max_features': ['sqrt','log2'],'min_samples_split': [2,10], 'n_jobs': [-1]},
    'LR': { 'penalty': ['l1','l2'], 'C': [0.00001,0.001,0.1,1,10]},
    'SGD': { 'loss': ['hinge','log','perceptron'], 'penalty': ['l2','l1','elasticnet']},
    'ET': { 'n_estimators': [10,100], 'criterion' : ['gini', 'entropy'] ,'max_depth': [5,50], 'max_features': ['sqrt','log2'],'min_samples_split': [2,10], 'n_jobs': [-1]},
    'AB': { 'algorithm': ['SAMME', 'SAMME.R'], 'n_estimators': [1,10,100,1000,10000]},
    'GB': {'n_estimators': [10,100], 'learning_rate' : [0.001,0.1,0.5],'subsample' : [0.1,0.5,1.0], 'max_depth': [5,50]},
    'NB' : {},
    'DT': {'criterion': ['gini', 'entropy'], 'max_depth': [1,5,10,20,50,100],'min_samples_split': [2,5,10]},
    'SVM' :{'C' :[0.00001,0.0001,0.001,0.01,0.1,1,10],'kernel':['linear']},
    'KNN' :{'n_neighbors': [1,5,10,25,50,100],'weights': ['uniform','distance'],'algorithm': ['auto','ball_tree','kd_tree']}
           }
    
    test_grid = { 
    'RF':{'n_estimators': [1], 'max_depth': [1], 'max_features': ['sqrt'],'min_samples_split': [10]},
    'LR': { 'penalty': ['l1'], 'C': [0.01]},
    'SGD': { 'loss': ['perceptron'], 'penalty': ['l2']},
    'ET': { 'n_estimators': [1], 'criterion' : ['gini'] ,'max_depth': [1], 'max_features': ['sqrt'],'min_samples_split': [10]},
    'AB': { 'algorithm': ['SAMME'], 'n_estimators': [1]},
    'GB': {'n_estimators': [1], 'learning_rate' : [0.1],'subsample' : [0.5], 'max_depth': [1]},
    'NB' : {},
    'DT': {'criterion': ['gini'], 'max_depth': [1],'min_samples_split': [10]},
    'SVM' :{'C' :[0.01],'kernel':['linear']},
    'KNN' :{'n_neighbors': [5],'weights': ['uniform'],'algorithm': ['auto']}
           }
    
    if (grid_size == 'large'):
        return clfs, large_grid
    elif (grid_size == 'small'):
        return clfs, small_grid
    elif (grid_size == 'test'):
        return clfs, test_grid
    else:
        return 0, 0

# a set of helper function to do machine learning evalaution

def joint_sort_descending(l1, l2):
    # l1 and l2 have to be numpy arrays
    idx = np.argsort(l1)[::-1]
    return l1[idx], l2[idx]

def generate_binary_at_k(y_scores, k):
    cutoff_index = int(len(y_scores) * (k / 100.0))
    test_predictions_binary = [1 if x < cutoff_index else 0 for x in range(len(y_scores))]
    return test_predictions_binary

def precision_at_k(y_true, y_scores, k):
    y_scores, y_true = joint_sort_descending(np.array(y_scores), np.array(y_true))
    preds_at_k = generate_binary_at_k(y_scores, k)
    #precision, _, _, _ = metrics.precision_recall_fscore_support(y_true, preds_at_k)
    #precision = precision[1]  # only interested in precision for label 1
    precision = precision_score(y_true, preds_at_k)
    return precision

def recall_at_k(y_true, y_scores, k):
    #y_scores_sorted, y_true_sorted = zip(*sorted(zip(y_scores, y_true), reverse=True))
    y_scores_sorted, y_true_sorted = joint_sort_descending(np.array(y_scores), np.array(y_true))
    preds_at_k = generate_binary_at_k(y_scores_sorted, k)
    #precision, _, _, _ = metrics.precision_recall_fscore_support(y_true, preds_at_k)
    #precision = precision[1]  # only interested in precision for label 1
    recall = recall_score(y_true_sorted, preds_at_k)
    return recall

def plot_precision_recall_n(y_true, y_prob, model_name):
    from sklearn.metrics import precision_recall_curve
    y_score = y_prob
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true, y_score)
    precision_curve = precision_curve[:-1]
    recall_curve = recall_curve[:-1]
    pct_above_per_thresh = []
    number_scored = len(y_score)
    for value in pr_thresholds:
        num_above_thresh = len(y_score[y_score>=value])
        pct_above_thresh = num_above_thresh / float(number_scored)
        pct_above_per_thresh.append(pct_above_thresh)
    pct_above_per_thresh = np.array(pct_above_per_thresh)
    
    plt.clf()
    fig, ax1 = plt.subplots()
    ax1.plot(pct_above_per_thresh, precision_curve, 'b')
    ax1.set_xlabel('percent of population')
    ax1.set_ylabel('precision', color='b')
    ax2 = ax1.twinx()
    ax2.plot(pct_above_per_thresh, recall_curve, 'r')
    ax2.set_ylabel('recall', color='r')
    ax1.set_ylim([0,1])
    ax1.set_ylim([0,1])
    ax2.set_xlim([0,1])
    
    name = model_name
    plt.title(name)
    #plt.savefig(name)
    plt.show()
    


def clf_loop(models_to_run, clfs, grid, X_train, X_test, y_train, y_test):
    """Runs the loop using models_to_run, clfs, gridm and the data
    """
    results_df =  pd.DataFrame(columns=('model_type','clf', 'parameters', 'auc-roc','baseline', 'p_at_1','p_at_2','p_at_5', 'p_at_10', 'p_at_20','p_at_30','p_at_50','recall_at_1','recall_at_2','recall_at_5','recall_at_10','recall_at_20','recall_at_30','recall_at_50','F1_at_1','F1_at_2','F1_at_5','F1_at_10','F1_at_20','F1_at_30','F1_at_50'))
    
    for n in range(1, 2):
        # create training and valdation sets
        for index,clf in enumerate([clfs[x] for x in models_to_run]):
            print(models_to_run[index])
            parameter_values = grid[models_to_run[index]]
            for p in ParameterGrid(parameter_values):
                try:
                    clf.set_params(**p)
                    y_pred_probs = clf.fit(X_train, y_train).predict_proba(X_test)[:,1]
                    # you can also store the model, feature importances, and prediction scores
                    # we're only storing the metrics for now
                    y_pred_probs_sorted, y_test_sorted = zip(*sorted(zip(y_pred_probs, y_test), reverse=True))
                    results_df.loc[len(results_df)] = [models_to_run[index],clf, p,
                                                       roc_auc_score(y_test, y_pred_probs),
                                                       precision_at_k(y_test_sorted,y_pred_probs_sorted,100.0),
                                                       precision_at_k(y_test_sorted,y_pred_probs_sorted,1.0),
                                                       precision_at_k(y_test_sorted,y_pred_probs_sorted,2.0),
                                                       precision_at_k(y_test_sorted,y_pred_probs_sorted,5.0),
                                                       precision_at_k(y_test_sorted,y_pred_probs_sorted,10.0),
                                                       precision_at_k(y_test_sorted,y_pred_probs_sorted,20.0),
                                                       precision_at_k(y_test_sorted,y_pred_probs_sorted,30.0),
                                                       precision_at_k(y_test_sorted,y_pred_probs_sorted,50.0),
                                                       recall_at_k(y_test_sorted,y_pred_probs_sorted, 1.0),
                                                       recall_at_k(y_test_sorted,y_pred_probs_sorted, 2.0),
                                                       recall_at_k(y_test_sorted,y_pred_probs_sorted, 5.0),
                                                       recall_at_k(y_test_sorted,y_pred_probs_sorted, 10.0),
                                                       recall_at_k(y_test_sorted,y_pred_probs_sorted, 20.0),
                                                       recall_at_k(y_test_sorted,y_pred_probs_sorted, 30.0),
                                                       recall_at_k(y_test_sorted,y_pred_probs_sorted, 50.0),
                                                       2*(precision_at_k(y_test_sorted,y_pred_probs_sorted,1.0)*recall_at_k(y_test_sorted,y_pred_probs_sorted, 1.0))/(precision_at_k(y_test_sorted,y_pred_probs_sorted,1.0) + recall_at_k(y_test_sorted,y_pred_probs_sorted, 1.0)),
                                                       2*(precision_at_k(y_test_sorted,y_pred_probs_sorted,2.0)*recall_at_k(y_test_sorted,y_pred_probs_sorted, 2.0))/(precision_at_k(y_test_sorted,y_pred_probs_sorted,2.0) + recall_at_k(y_test_sorted,y_pred_probs_sorted, 2.0)),
                                                      2*(precision_at_k(y_test_sorted,y_pred_probs_sorted,5.0)*recall_at_k(y_test_sorted,y_pred_probs_sorted, 5.0))/(precision_at_k(y_test_sorted,y_pred_probs_sorted,5.0) + recall_at_k(y_test_sorted,y_pred_probs_sorted, 5.0)),
                                                      2*(precision_at_k(y_test_sorted,y_pred_probs_sorted,10.0)*recall_at_k(y_test_sorted,y_pred_probs_sorted, 10.0))/(precision_at_k(y_test_sorted,y_pred_probs_sorted,10.0) + recall_at_k(y_test_sorted,y_pred_probs_sorted, 10.0)),
                                                      2*(precision_at_k(y_test_sorted,y_pred_probs_sorted,20.0)*recall_at_k(y_test_sorted,y_pred_probs_sorted, 20.0))/(precision_at_k(y_test_sorted,y_pred_probs_sorted,20.0) + recall_at_k(y_test_sorted,y_pred_probs_sorted, 20.0)),
                                                      2*(precision_at_k(y_test_sorted,y_pred_probs_sorted,30.0)*recall_at_k(y_test_sorted,y_pred_probs_sorted, 30.0))/(precision_at_k(y_test_sorted,y_pred_probs_sorted,30.0) + recall_at_k(y_test_sorted,y_pred_probs_sorted, 30.0)),
                                                      2*(precision_at_k(y_test_sorted,y_pred_probs_sorted,50.0)*recall_at_k(y_test_sorted,y_pred_probs_sorted, 50.0))/(precision_at_k(y_test_sorted,y_pred_probs_sorted,50.0) + recall_at_k(y_test_sorted,y_pred_probs_sorted, 50.0))]
                    if NOTEBOOK == 1:
                        plot_precision_recall_n(y_test,y_pred_probs,clf)
                except IndexError as e:
                    print('Error:',e)
                    continue
    return results_df

In [46]:

def LR():
    return LogisticRegression(penalty = 'l1', C = 1e5)

def KNN():
    return KNeighborsClassifier(n_neighbors = 3)

def DT():
    return DecisionTreeClassifier()

def SVM():
    return svm.SVC(kernel = 'linear', probability = True, random_state = 3)

def RF():
    return RandomForestClassifier(n_estimators = 50, n_jobs = -1)

def AB():
    return AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                                                    algorithm="SAMME",
                                                    n_estimators=200)
def GB():
    return GradientBoostingClassifier(learning_rate = 0.05,
                                    	subsample = 0.5,
                                    	max_depth = 6,
                                    	n_estimators = 10)

In [20]:
mlf.convert_true_false_1_0(mh_df1)

In [21]:
mh_df1.columns

Index(['dedupe_id', 'sex', 'dob', 'age_calced', 'race', 'mar_stat', 'city',
       'zip', 'rel_date', 'arr_agency', 'arrest_dt', 'arrest_age', 'bk_dt',
       'unbooked_days', 'stay_days', 'patid', 'mh_stay_days', 'program',
       'pri_dx_value', 'admit_date', 'dschg_date', 'discharge_reason',
       'dx_description', 'dx_date', 'anxiety_dummy', 'depression_dummy',
       'psychotic_dummy', 'ptsd_dummy', 'opps_dummy', 'drugs_dummy'],
      dtype='object')

In [22]:
x = mh_df1.head(20)
x.style

,dedupe_id,sex,dob,age_calced,race,mar_stat,city,zip,rel_date,arr_agency,arrest_dt,arrest_age,bk_dt,unbooked_days,stay_days,patid,mh_stay_days,program,pri_dx_value,admit_date,dschg_date,discharge_reason,dx_description,dx_date,anxiety_dummy,depression_dummy,psychotic_dummy,ptsd_dummy,opps_dummy,drugs_dummy
0,2,MALE,1955-08-25,54,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0
1,3,MALE,1962-03-18,47,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0
2,4,FEMALE,1976-09-07,33,WHITE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0
3,5,FEMALE,1980-05-27,29,WHITE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,137277,339,OUTPATIENT,DYSTHYMIA,nan,nan,nan,DYSTHYMIC DISORDER,2012-09-07,0,0,0,0,0,0
4,6,MALE,1937-08-14,72,WHITE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0
5,7,FEMALE,1951-11-07,58,WHITE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0
6,8,FEMALE,1977-12-22,32,WHITE,S,KANSAS CITY,66102,2003-04-16,KS0460600,2003-04-15,25,2003-04-15,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0
7,8,FEMALE,1977-12-22,32,WHITE,nan,KANSAS CITY,64131,2001-08-11,KS0460600,2001-08-11,23,2001-08-11,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0
8,8,FEMALE,1977-12-22,32,WHITE,S,KANSAS CITY,66102,2003-02-03,KS0460600,2003-02-03,25,2003-02-03,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0
9,8,FEMALE,1977-12-22,32,WHITE,S,KANSAS CITY,66102,2002-03-27,KS0460600,2002-03-26,24,2002-03-26,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0


In [23]:
test_df = mh_df1.head(100)

In [24]:
'''
def label_maker(df, label, delta = 1,):
    df[label] = 0
    for idx, row in df.iterrows():
        #print(row['rel_date'])
        try:
            r = datetime.strptime(row['rel_date'], '%Y-%m-%d')
            t = r + relativedelta(years=+delta)
            d_id = row['dedupe_id']
            for idx2, row2 in df.iterrows():
                if d_id == row2.dedupe_id and datetime.strptime(row2.bk_dt, '%Y-%m-%d') < t and datetime.strptime(row2.rel_date, '%Y-%m-%d') > r:
                    df.loc[idx, label] = 1
        except Exception as ex:
            pass
'''

"\ndef label_maker(df, label, delta = 1,):\n    df[label] = 0\n    for idx, row in df.iterrows():\n        #print(row['rel_date'])\n        try:\n            r = datetime.strptime(row['rel_date'], '%Y-%m-%d')\n            t = r + relativedelta(years=+delta)\n            d_id = row['dedupe_id']\n            for idx2, row2 in df.iterrows():\n                if d_id == row2.dedupe_id and datetime.strptime(row2.bk_dt, '%Y-%m-%d') < t and datetime.strptime(row2.rel_date, '%Y-%m-%d') > r:\n                    df.loc[idx, label] = 1\n        except Exception as ex:\n            pass\n"

In [25]:
def isNaN(num):
    return num != num

def label_maker(df, label='returner', delta = 1):
    
    df[label] = 0
    
    for i, row_i in df.iterrows():
        
        if isNaN(row_i.rel_date) == False:
            current_booking = datetime.strptime(row_i.rel_date, '%Y-%m-%d')
            time_limit = current_booking + relativedelta(years=+delta)
            d_id = row_i.dedupe_id
            
            for j, row_j in df.iterrows():
                if isNaN(row_j.bk_dt) == False and isNaN(row_j.rel_date) == False:
                    booking_date = datetime.strptime(row_j.bk_dt, '%Y-%m-%d')
                    release_date = datetime.strptime(row_j.rel_date, '%Y-%m-%d')
                    
                    if d_id == row_j.dedupe_id and release_date > current_booking and booking_date < time_limit:
                        df.loc[i, label] = 1

In [26]:
label_maker(test_df, 'returner')

In [27]:
test_df.style

,dedupe_id,sex,dob,age_calced,race,mar_stat,city,zip,rel_date,arr_agency,arrest_dt,arrest_age,bk_dt,unbooked_days,stay_days,patid,mh_stay_days,program,pri_dx_value,admit_date,dschg_date,discharge_reason,dx_description,dx_date,anxiety_dummy,depression_dummy,psychotic_dummy,ptsd_dummy,opps_dummy,drugs_dummy,returner
0,2,MALE,1955-08-25,54,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0
1,3,MALE,1962-03-18,47,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0
2,4,FEMALE,1976-09-07,33,WHITE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0
3,5,FEMALE,1980-05-27,29,WHITE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,137277,339,OUTPATIENT,DYSTHYMIA,nan,nan,nan,DYSTHYMIC DISORDER,2012-09-07,0,0,0,0,0,0,0
4,6,MALE,1937-08-14,72,WHITE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0
5,7,FEMALE,1951-11-07,58,WHITE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0
6,8,FEMALE,1977-12-22,32,WHITE,S,KANSAS CITY,66102,2003-04-16,KS0460600,2003-04-15,25,2003-04-15,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,1
7,8,FEMALE,1977-12-22,32,WHITE,nan,KANSAS CITY,64131,2001-08-11,KS0460600,2001-08-11,23,2001-08-11,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,1
8,8,FEMALE,1977-12-22,32,WHITE,S,KANSAS CITY,66102,2003-02-03,KS0460600,2003-02-03,25,2003-02-03,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,1
9,8,FEMALE,1977-12-22,32,WHITE,S,KANSAS CITY,66102,2002-03-27,KS0460600,2002-03-26,24,2002-03-26,0,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,1


In [57]:
relevant_df = test_df[['returner',
       'anxiety_dummy', 'depression_dummy', 'psychotic_dummy', 'ptsd_dummy',
       'opps_dummy', 'drugs_dummy']]

In [58]:
relevant_df['returner'].unique()

array([0, 1])

In [65]:
x_train, x_test, y_train, y_test  = mlf.train_test_data_split(relevant_df, 'returner', .2)

In [66]:
# Modified based on Rayid's magic loop

def run_simple_loop(x_train, x_test, y_train, y_test):

    # define grid to use: test, small, large
    grid_size = 'test'
    clfs, grid = define_clfs_params(grid_size)
    
    # define models to run
    models_to_run=['RF','DT','KNN', 'ET', 'AB', 'GB', 'LR', 'NB']

    # call clf_loop and store results in results_df
    results_df = clf_loop(models_to_run, clfs,grid, x_train, x_test, y_train, y_test)

    return results_df

In [69]:
run_simple_loop(x_train, x_test, y_train, y_test)

RF
DT
KNN
ET
AB
GB
LR
NB


,model_type,clf,parameters,auc-roc,baseline,p_at_1,p_at_2,p_at_5,p_at_10,p_at_20,...,recall_at_20,recall_at_30,recall_at_50,F1_at_1,F1_at_2,F1_at_5,F1_at_10,F1_at_20,F1_at_30,F1_at_50
0,RF,"(DecisionTreeClassifier(class_weight=None, cri...","{'max_depth': 1, 'max_features': 'sqrt', 'min_...",1.0,0.05,0.0,0.0,1.0,0.5,0.25,...,1.0,1.0,1.0,NaN,NaN,1.0,0.666667,0.4,0.285714,0.181818
1,DT,"DecisionTreeClassifier(class_weight=None, crit...","{'criterion': 'gini', 'max_depth': 1, 'min_sam...",1.0,0.05,0.0,0.0,1.0,0.5,0.25,...,1.0,1.0,1.0,NaN,NaN,1.0,0.666667,0.4,0.285714,0.181818
2,KNN,"KNeighborsClassifier(algorithm='auto', leaf_si...","{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",1.0,0.05,0.0,0.0,1.0,0.5,0.25,...,1.0,1.0,1.0,NaN,NaN,1.0,0.666667,0.4,0.285714,0.181818
3,ET,"(ExtraTreeClassifier(class_weight=None, criter...","{'criterion': 'gini', 'max_depth': 1, 'max_fea...",1.0,0.05,0.0,0.0,1.0,0.5,0.25,...,1.0,1.0,1.0,NaN,NaN,1.0,0.666667,0.4,0.285714,0.181818
4,AB,"(DecisionTreeClassifier(class_weight=None, cri...","{'algorithm': 'SAMME', 'n_estimators': 1}",1.0,0.05,0.0,0.0,1.0,0.5,0.25,...,1.0,1.0,1.0,NaN,NaN,1.0,0.666667,0.4,0.285714,0.181818
5,GB,([DecisionTreeRegressor(criterion='friedman_ms...,"{'learning_rate': 0.1, 'max_depth': 1, 'n_esti...",1.0,0.05,0.0,0.0,1.0,0.5,0.25,...,1.0,1.0,1.0,NaN,NaN,1.0,0.666667,0.4,0.285714,0.181818
6,LR,"LogisticRegression(C=0.01, class_weight=None, ...","{'C': 0.01, 'penalty': 'l1'}",0.5,0.05,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NB,GaussianNB(priors=None),{},1.0,0.05,0.0,0.0,1.0,0.5,0.25,...,1.0,1.0,1.0,NaN,NaN,1.0,0.666667,0.4,0.285714,0.181818
